In [1]:
import torch

import numpy as np
import pandas as pd
from models.mlp import MLP

from tqdm import tqdm

from torch.utils.data.dataset import TensorDataset

In [2]:
train = pd.read_csv("./dataset/exported/data_176_cleaned/train.csv")
test = pd.read_csv("./dataset/exported/data_176_cleaned/test.csv")

all_data = pd.concat([train, test], axis=0)

In [3]:
real_led_6 = test["led_6"]

In [4]:
test["led_6"] = test["led_6"]*0.95

In [5]:
X = torch.from_numpy(test.drop(["x", "y"], axis="columns").to_numpy()).to(dtype=torch.float32)
y = torch.from_numpy(test[["x", "y"]].to_numpy()).to(dtype=torch.float32)

In [6]:
model = MLP(normalize=True, seed=42, epochs=25)
model.load("./saved_runs/MLP-TINY-NORMALISE-1747596375.pth")

In [7]:
predicted = ((model.predict(X) / 10).round()*10).to(dtype=torch.int64)

In [8]:
led_6 = 0
n = 0
random_idx = np.random.randint(0, len(test), 1000)
for i in tqdm(range(1000)):
    i = random_idx[i]
    predicted_i = predicted[i, :]
    
    first = all_data[all_data["x"] == predicted_i[0].item()]
    second = first[first["y"] == predicted_i[1].item()]
    
    if len(second["led_6"]) == 0 or X[i, 6] == 0:
        continue

    led_6 += second["led_6"].item() / X[i, 6]
    n += 1

predicted_scalar = led_6/n
print(f"Predicted scalar: {predicted_scalar}")

100%|██████████| 1000/1000 [00:00<00:00, 1078.62it/s]

Predicted scalar: 1.0157092809677124


In [9]:
adjusted_X = X.clone()
adjusted_X[:, 6] = adjusted_X[:, 6] * predicted_scalar

In [10]:
asjusted_predicted = ((model.predict(adjusted_X) / 10).round()*10).to(dtype=torch.int64)

In [12]:
led_6 = 0
n = 0
for i in tqdm(range(1000)):
    i = random_idx[i]
    predicted_i = asjusted_predicted[i, :]
    
    first = all_data[all_data["x"] == predicted_i[0].item()]
    second = first[first["y"] == predicted_i[1].item()]
    
    if len(second["led_6"]) == 0 or adjusted_X[i, 6] == 0:
        continue

    led_6 += second["led_6"].item() / adjusted_X[i, 6]
    n += 1

adjusted_predicted_scalar = led_6/n
print(f"After adjusting predicted scalar: {adjusted_predicted_scalar}")

100%|██████████| 1000/1000 [00:00<00:00, 1089.21it/s]

After adjusting predicted scalar: 0.9972536563873291
